# Lecture: Witness Sets

In [46]:
using HomotopyContinuation

So far, we've seen that numerical methods are very good at solving **zero-dimensional polynomial systems**

**Question**: What do we do if we are interested in a **positive-dimensional** set of solutions?

**Answer**: Slice it with enough hyperplanes until we have a zero-dimensional solution set.



<img src="SimpleWitness.png" width="300">

The data in such a *slice* is called a **witness set**.

A witness set for an irreducible variety $X \subset \mathbb{C}^n$ is a triple 
$$W=(F,L,S)$$
where 
    
- $\color{blue}{F}$ is a polynomial system for which $X$ is a component, 
- $\color{green}{L}$ is a generic affine linear space of complementary dimension
- $\color{red}{S} = X \cap L$



<p style="border:3px; border-style:solid; padding: 0.5em; text-align:center">
How is this a good representation of the variety $X$?
</p>


Given a witness set $W=(F,L,S)$ for $X$...

Certain information about $X$ become obvious:
- dimension (equals the codimension of $L$)
- degree (equals the number of points in $S$)

Certain algorithms become almost trivial

- Membership: is $p$ contained in $X$?

- Sampling: find $100000$ points on $X$

- Find a (pseudo) witness set for a projection of $X$

One major advantage (compared to symbolic representations) is that they can be computed fast. 

One major disadvantage is that it can be difficult to prove things (but not impossible)

<hr style="border:1px solid gray"> </hr>

## Constructing a witness set using homotopycontinuation.jl

In [47]:
@var x y z
TwistedCubic=System([y^2-x*z,x*y-z,x^2-y])

System of length 3
 3 variables: x, y, z

 -x*z + y^2
 -z + x*y
 -y + x^2

In [48]:
#Try the following without codimension: you get an error because it expects the dimension to be zero.
W_twisted=witness_set(TwistedCubic; codim=2)

Witness set for dimension 1 of degree 3

In [49]:
#Degree of the twisted cubic
degree(W_twisted)

3

In [50]:
#Dimension of the twisted cubic
dim(W_twisted)

1

In [51]:
#Generic linear space used to slice the twisted cubic
W_twisted.L

2-dim. affine linear subspace {x | Ax=b} with eltype Complex{Float64}:
A:
Complex{Float64}[-0.48978411425480606 + 0.02580177253433902im 0.21613924159631848 - 0.27938234668321293im 0.0636065099789099 + 0.794121737802192im]
b:
Complex{Float64}[-0.06499561163805319 - 0.41533725552535355im]

In [52]:
#Witness points
solutions(W_twisted)

3-element Array{Array{Complex{Float64},1},1}:
 [-0.6759341506362687 + 0.3314467193843036im, 0.34703004820575656 - 0.4480723134964138im, -0.08605736252389991 + 0.41788934965240115im]
 [0.7100974999955674 - 0.6816912560560303im, 0.039535490916706544 - 0.9681345133884506im, -0.631894779201916 - 0.7144208960783656im]
 [0.29374572480119804 + 0.6486829021167949im, -0.3345029566596861 + 0.38109565849688504im, -0.3454690511900137 - 0.10504112826889644im]

## Some simple algorithms involving witness sets

### Moving the linear space (this is the fundamental operation)

In [53]:
L2 = LinearSubspace([1,2,3]',[5])

2-dim. affine linear subspace {x | Ax=b} with eltype Float64:
A:
[-0.2672612419124243 -0.5345224838248488 -0.8017837257372732]
b:
[-1.3363062095621219]

In [54]:
W_twisted2=witness_set(W_twisted,L2)

Witness set for dimension 1 of degree 3

In [55]:
solutions(W_twisted2)

3-element Array{Array{Complex{Float64},1},1}:
 [-0.7954052524310862 + 1.0820358328370792im, -0.5381320279484715 - 1.7213139695145148im, 2.290556436109343 + 0.786864035397317im]
 [-0.7954052524310862 - 1.0820358328370792im, -0.5381320279484715 + 1.7213139695145148im, 2.290556436109343 - 0.786864035397317im]
 [0.9241438381955056 + 7.222237291452134e-35im, 0.8540418336747209 + 1.2037062152420224e-34im, 0.7892574981516842 + 2.407412430484045e-35im]

### Sampling

In [56]:
function sample(W::WitnessSet)
    n=length(solutions(W)[1]) #number of variables
    randomLinear = LinearSubspace(randn(ComplexF64,dim(W),n),randn(ComplexF64,dim(W)))
    newWitness=witness_set(W,randomLinear)
    s=solutions(newWitness)[1]
    return s
end

sample (generic function with 1 method)

In [57]:
MySamples=[sample(W_twisted) for i in 1:100]

100-element Array{Array{Complex{Float64},1},1}:
 [0.055140952352266806 + 1.6323054720155636im, -2.661380629345637 + 0.18001375651350912im, -0.4405885022690718 - 4.3342600344264435im]
 [-0.2517091698380603 - 0.19399660181921383im, 0.02572282466316289 + 0.0976614471906381im, 0.012471318041875682 - 0.029572422371384147im]
 [0.06258287760988822 + 1.379051632321684im, -1.8978667880391669 + 0.17261003904660902im, -0.3568121210180626 - 2.6064538590266326im]
 [-1.9653482343708975 + 0.6659636241339104im, 3.419086133675232 - 2.6177008656936493im, -4.976401340564817 + 7.421680767310541im]
 [-1.2193913077223713 - 0.49580431513331136im, 1.2410932424440633 + 1.2091589444096065im, -0.9138720895889417 - 2.0897772915543915im]
 [-1.236127144070392 - 1.7538071521995253im, -1.5478292107985858 + 4.335857252597253im, 9.51757116237443 - 2.645076902468032im]
 [0.9836063697951519 + 1.3959141850530383im, -0.981094921330691 + 2.746060168211154im, -4.7982755558096235 + 1.331517955724231im]
 [-0.5467813154974943 -

### Membership

In [58]:
function membership(W::WitnessSet,p)
    n=length(solutions(W)[1]) #number of variables
    A=randn(ComplexF64,dim(W),n)
    randomLinearThroughP = LinearSubspace(A,A*p)
    newWitness=witness_set(W,randomLinearThroughP)
    check=findfirst(x->x≈p,solutions(newWitness))
    if check==nothing
        return false
    else
        return true
    end
end
    

membership (generic function with 1 method)

In [59]:
membership(W_twisted,[2,4,8])

true

In [60]:
membership(W_twisted,[2,5,8])

false

In [61]:
[membership(W_twisted,s) for s in MySamples]

100-element Array{Bool,1}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1

## Pseudo witness sets

Both the sampling and membership algorithms (and most other algorithms involving witness sets) depend on one thing:


**the ability to move the linear space of the witness set and track the witness points**

Observation: we can do this for projections trivially

**Definition:** A pseudo witness set for an (irreducible) variety $X\subset \mathbb{C}^n$ is a quadruple 
$$(F,\pi,\pi^{-1}(L),S)$$
where 
- $\pi:\mathbb{C}^N \to \mathbb{C}^n$ (really this works with any map by factoring through the graph)
- $X$ is a component of $\pi(\mathcal{V}(F))$ of the same dimension as $\mathcal{V}(F)$
- $L$ is a linear space of complementary dimension (to $X$) in $\mathbb{C}^n$
- $S = \pi^{-1}(L) \cap \mathcal V(F)$ (note! this is in $\mathbb{C}^N$)

<img src="PseudoWitness1.png" width="300">

As the picture above shows, $\pi^{-1}(L)$ may not be generic for $\mathcal{V}(F)$ even though $L$ is generic for $X$


**Warning** be careful about the following situation

<img src="Careful.png" width="200">

In [62]:
PseudoWitness_Parabola=witness_set(W_twisted,LinearSubspace([3,6,0]',[1]))

Witness set for dimension 1 of degree 2

In [63]:
ProjectedSolutions=[s[1:2] for s in solutions(PseudoWitness_Parabola)]

2-element Array{Array{Complex{Float64},1},1}:
 [-0.728713553878169 + 0.0im, 0.5310234436057513 + 0.0im]
 [0.22871355387816908 - 7.52316384526264e-37im, 0.05230988972758215 + 0.0im]

In [64]:
Witness_Parabola=witness_set(System([y-x^2])) #computes a witness set for the parabola
Witness_Parabola2=witness_set(Witness_Parabola,LinearSubspace([3,6]',[1])) #moves it to the space we used above

Witness set for dimension 1 of degree 2

In [65]:
solutions(Witness_Parabola2)

2-element Array{Array{Complex{Float64},1},1}:
 [0.22871355387816908 + 3.587324068671532e-43im, 0.05230988972758215 + 8.96831017167883e-44im]
 [-0.728713553878169 + 9.62964972193618e-35im, 0.5310234436057513 - 9.62964972193618e-35im]

Membership/sampling can be easily coded for these pseudo-witness sets as well

## Witness supersets, monodromy, the trace test, and irreducible decomposition

In [13]:
#If we only take two of the generators, we get a complete intersection of the union of a cubic and a line
CompleteIntersection=System([y^2-x*z,x*y-z])
W_complete=witness_set(CompleteIntersection)

Witness set for dimension 1 of degree 4


<img src="witnesscubic.png" width="200">

<p style="border:3px; border-style:solid; padding: 0.5em; text-align:center">
Warning: W_complete is not a witness set for the twisted cubic.
</p>

A **witness superset** for $\mathcal{V}(f_1,\ldots,f_k)$ in codimension $m$ is the triple $W=(F,L,S)$
    
- $\color{blue}{F}$ is a polynomial system
- $\color{green}{L}$ is a generic affine linear space of dimension $m$
- $\color{red}{S} = X \cap L$


Let $X \subseteq \mathbb{C}^n$ be an (equidimensional) variety of codimension $m$.

We have a branched cover 


$$Z = \{(x,L) \in \mathbb{C}^n \times \textrm{Gr}(m+1,n+1) \mid x \in X \cap L \} \subset \mathbb{C}^n \times \textrm{Gr}(m+1,n+1)$$
$$\,\,\,\, \downarrow \pi $$
$$\textrm{Gr}(m+1,n+1)$$



**Theorem**: If $X$ is irreducible then the monodromy group of $\pi$ is the full symmetric group.

**Corollary**: If $X=X_1,\ldots,X_k$ is equidimensional, then the monodromy group of $\pi$  is the product of symmetric groups $\mathfrak{S}_{\textrm{deg}(X_1)} \times \cdots \times \mathfrak{S}_{\textrm{deg}(X_k)}$

Let's try to compute the witness points using monodromy

In [67]:
M=monodromy_solve(CompleteIntersection; dim=1) 

MonodromyResult
• return_code → :success
• 3 solutions
• 12 tracked loops
• random_seed → 0x002f490d
• trace → 5.444635406357445e-17

In [72]:
#Trying this many times will give us either 1 or 3 solutions (possibly 2 if the heuristic stop didn't work)
[length(solutions(monodromy_solve(CompleteIntersection;dim=1))) for i in 1:5]

5-element Array{Int64,1}:
 1
 1
 1
 3
 1

**Monodromy**: allows us to find points on the same component

**The trace test**: tells us when we've found all points on the same component


The **trace** of a subset of points in $\mathbb{C}^n$ is their coordinate-wise average.

**Lemma**: Let $X$ be irreducible. The trace of a subset of witness points of $X$ moves linearly if and only if the subset is not proper.


<img src="TraceExample1.png" width="600">

Now let's try moving a witness set and verifying this ourselves

In [96]:
W_complete

Witness set for dimension 1 of degree 4

In [104]:
A=randn(ComplexF64,1,3)
b=rand(ComplexF64)
L₁ = LinearSubspace(A,[b])
L₂ = LinearSubspace(A,[b+1])
L₃ = LinearSubspace(A,[b+2]);

In [105]:
trace₁=sum(solutions(witness_set(W_complete,L₁)))
println("First trace: \n",trace₁)
trace₂=sum(solutions(witness_set(W_complete,L₂)))
println("Second trace: \n",trace₂)
trace₃=sum(solutions(witness_set(W_complete,L₃)))
println("Third trace: \n",trace₃)

First trace: 
Complex{Float64}[5.465329287538794 - 3.2635780624323534im, 19.158332630134044 - 35.07012400164239im, 7.32701915075733 - 239.46249925237703im]
Second trace: 
Complex{Float64}[5.930456986784706 - 4.439466156969887im, 19.15833263013405 - 35.07012400164239im, 24.739910843035197 - 230.7778466264806im]
Third trace: 
Complex{Float64}[6.395584686030619 - 5.615354251507423im, 19.158332630134044 - 35.0701240016424im, 42.152802535312965 - 222.0931940005842im]


In [106]:
# here we check trace2 is the midpoint of trace1 and trace3
(trace₁+trace₃)-2*trace₂

3-element Array{Complex{Float64},1}:
  1.7763568394002505e-15 - 1.7763568394002505e-15im
 -1.4210854715202004e-14 - 1.4210854715202004e-14im
  -9.947598300641403e-14 + 0.0im

In [172]:
trace_test(W_complete)

2.0617248710910148e-17

We can find out which solution is the one on the line by checking traces of subsets

In [173]:
I=[1]
trace₁=sum(solutions(witness_set(W_complete,L₁))[I])
trace₂=sum(solutions(witness_set(W_complete,L₂))[I])
trace₃=sum(solutions(witness_set(W_complete,L₃))[I])
# here we check trace2 is the midpoint of trace1 and trace 3
(trace₁+trace₃)-2*trace₂

3-element Array{Complex{Float64},1}:
  2.220446049250313e-16 + 0.0im
 1.542348713665846e-179 - 3.855871784164615e-180im
                    0.0 + 0.0im

In [116]:
[membership(W_twisted,s) for s in solutions(W_complete)]

4-element Array{Bool,1}:
 0
 1
 1
 1

### Numerical irreducible decomposition (of equidimensional varieties)

One of the most important algorithms in numerical algebraic geometry is taking a polynomial systme $F$ and computing a witness set for every irreducible component of $\mathcal V(F)$. This is called a **numerical irreducible decomposition**. We will go through how it works for equidimensional varieties

#### Step 1: compute a witness superset 

$\mathcal V(F) \to (F,L,S)$

#### Step 2: 

Using monodromy, identify solutions as belonging to the same component until the trace test passes.

$p \in S \to S_p=\{q \in S \mid q \text{ was found via monodromy starting from } p\}$

#### Step 3: 

If $S \backslash \bigcup_{p \text{ done in step 2}} S_p \neq \emptyset$ pick some $q$ not identified and repeat step 2

#### Output: 

Set $W_p = (F,L,S_p)$ and output the union of all $W_p$


### For images of maps

If $\varphi:\mathbb{C}_y^d \to \mathbb{C}_x^n$ is some map, you can compute a pseudo witness set for $\overline{\varphi(\mathbb{C}^d)}$ by taking $F = (x_1-\varphi_1(y),\ldots,x_n-\varphi_n(y))$ and $\pi:\mathbb{C}^{d+n} \to \mathbb{C}^n$ sending $\pi(y,x)=x$.

Note: If you've studied symbolic computational algebraic geometry, this is elimination...which is hard

In [155]:
W_parabola = witness_set(System([y-x^2]))

Tracking 2 paths... 100%|███████████████████████████████| Time: 0:00:02
  # paths tracked:                  2
  # non-singular solutions (real):  2 (0)
  # singular endpoints (real):      0 (0)
  # total solutions (real):         2 (0)


Witness set for dimension 1 of degree 2

The parabola is parametrized by the first two coordinates of the twisted cubic

In [161]:
println(W_parabola.L.extrinsic.A)
Slice_For_Cubic=LinearSubspace(hcat(W_parabola.L.extrinsic.A,[0]),W_parabola.L.extrinsic.b)
PseudoWitness=witness_set(W_twisted,Slice_For_Cubic)

Complex{Float64}[-0.649266982668212 + 0.5004085823698871im 0.5620539810837469 - 0.11017694066993688im]


Witness set for dimension 1 of degree 2

In [163]:
println([s[1:2] for s in solutions(PseudoWitness)])

Array{Complex{Float64},1}[[1.5834270985288863 - 1.131631007667107im, 1.2266526388419356 - 3.5837104061512948im], [-0.3029371480886612 + 0.49231854095540273im, -0.1506066300763651 - 0.2982831494964009im]]


In [164]:
println(solutions(W_parabola))

Array{Complex{Float64},1}[[1.5834270985288865 - 1.1316310076671072im, 1.2266526388419359 - 3.5837104061512957im], [-0.3029371480886612 + 0.49231854095540273im, -0.1506066300763651 - 0.2982831494964009im]]


You can move the witness set by moving the pseudo-witness set.

### Other things you can do with witness sets

Given a witness set $W$ for $X$ you can also...

- find a witness set for $X \cap \mathcal V(f)$
- recover equations for $X$
- compute the Newton polytope of $X$ (when $X$ is a hypersurface)
- describe the real points of $X$
- much more...